In [2]:
import h5pyd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.spatial import cKDTree

In [105]:
#J82JWhLzSODJvD9bnMurt7EV35121qTaI65ci6Ph
# Open the desired year of nsrdb data
# server endpoint, username, password is found via a config file
df = pd.read_csv ('./dataset/1060699_25.77_-80.18_2019.csv', skiprows=[0,1])
df.info()

df_header = pd.read_csv ('./dataset/1060699_25.77_-80.18_2019.csv',nrows=1)
df_header.info()
pd.set_option('display.max_columns', None)  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 25 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Year                                         17520 non-null  int64  
 1   Month                                        17520 non-null  int64  
 2   Day                                          17520 non-null  int64  
 3   Hour                                         17520 non-null  int64  
 4   Minute                                       17520 non-null  int64  
 5   DHI                                          17520 non-null  int64  
 6   DNI                                          17520 non-null  int64  
 7   GHI                                          17520 non-null  int64  
 8   Clearsky DHI                                 17520 non-null  int64  
 9   Clearsky DNI                                 17520 non-null  int64  
 10

In [106]:
df_header.head()

,Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,Clearsky DHI Units,Clearsky DNI Units,Clearsky GHI Units,Dew Point Units,DHI Units,DNI Units,GHI Units,Solar Zenith Angle Units,Temperature Units,Pressure Units,Relative Humidity Units,Precipitable Water Units,Wind Direction Units,Wind Speed,Cloud Type -15,Cloud Type 0,Cloud Type 1,Cloud Type 2,Cloud Type 3,Cloud Type 4,Cloud Type 5,Cloud Type 6,Cloud Type 7,Cloud Type 8,Cloud Type 9,Cloud Type 10,Cloud Type 11,Cloud Type 12,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Surface Albedo Units,Version
0,NSRDB,1060699,-,-,-,25.77,-80.18,-5,0,-5,w/m2,w/m2,w/m2,c,w/m2,w/m2,w/m2,Degree,c,mbar,%,cm,Degrees,m/s,NaN,Clear,Probably Clear,Fog,Water,Super-Cooled Water,Mixed,Opaque Ice,Cirrus,Overlapping,Overshooting,Unknown,Dust,Smoke,NaN,Missing Image,Low Irradiance,Exceeds Clearsky,Missing CLoud Properties,Rayleigh Violation,NaN,3.1.1


In [107]:
df.head()

,Year,Month,Day,Hour,Minute,DHI,DNI,GHI,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Solar Zenith Angle,Fill Flag,Surface Albedo,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure,Global Horizontal UV Irradiance (280-400nm),Global Horizontal UV Irradiance (295-385nm),Unnamed: 24
0,2019,1,1,0,0,0,0,0,0,0,0,0,21.8,173.88,0,0.08,3.7,2.4,128,87.10,24.1,1019,0.0,0.0,NaN
1,2019,1,1,0,30,0,0,0,0,0,0,0,21.8,176.95,0,0.08,3.6,2.4,128,87.10,24.1,1019,0.0,0.0,NaN
2,2019,1,1,1,0,0,0,0,0,0,0,0,21.7,171.38,0,0.08,3.6,2.4,127,86.53,24.1,1019,0.0,0.0,NaN
3,2019,1,1,1,30,0,0,0,0,0,0,0,21.7,164.76,0,0.08,3.5,2.4,126,86.53,24.1,1019,0.0,0.0,NaN
4,2019,1,1,2,0,0,0,0,0,0,0,0,21.5,158.02,0,0.08,3.5,2.4,126,85.94,24.0,1019,0.0,0.0,NaN


In [108]:
df = df[['Year','Month', 'Day', 'Hour', 'Minute','Surface Albedo','Cloud Type', 
    'Dew Point', 'Solar Zenith Angle', 'Pressure', 'Wind Direction', 'Wind Speed', 
    'Relative Humidity', 'Temperature', 'Precipitable Water', 'GHI']]
df.head()

,Year,Month,Day,Hour,Minute,Surface Albedo,Cloud Type,Dew Point,Solar Zenith Angle,Pressure,Wind Direction,Wind Speed,Relative Humidity,Temperature,Precipitable Water,GHI
0,2019,1,1,0,0,0.08,0,21.8,173.88,1019,128,3.7,87.10,24.1,2.4,0
1,2019,1,1,0,30,0.08,0,21.8,176.95,1019,128,3.6,87.10,24.1,2.4,0
2,2019,1,1,1,0,0.08,0,21.7,171.38,1019,127,3.6,86.53,24.1,2.4,0
3,2019,1,1,1,30,0.08,0,21.7,164.76,1019,126,3.5,86.53,24.1,2.4,0
4,2019,1,1,2,0,0.08,0,21.5,158.02,1019,126,3.5,85.94,24.0,2.4,0


In [109]:
df = df[(df['Hour'] < 22) & (df['Hour'] > 5)]
df.head()

,Year,Month,Day,Hour,Minute,Surface Albedo,Cloud Type,Dew Point,Solar Zenith Angle,Pressure,Wind Direction,Wind Speed,Relative Humidity,Temperature,Precipitable Water,GHI
12,2019,1,1,6,0,0.08,0,20.7,104.90,1019,125,4.0,82.54,23.9,2.4,0
13,2019,1,1,6,30,0.08,0,20.8,98.56,1020,126,4.1,82.13,24.0,2.4,0
14,2019,1,1,7,0,0.08,3,20.8,92.34,1020,127,4.3,81.93,24.1,2.4,2
15,2019,1,1,7,30,0.08,0,20.8,86.09,1020,129,4.5,80.00,24.5,2.4,45
16,2019,1,1,8,0,0.08,0,20.9,80.32,1020,130,4.8,78.72,24.9,2.4,90


In [110]:
df.shape

(11680, 16)